<a href="https://colab.research.google.com/github/dandaraaamorim/dandaraaamorim/blob/main/previs_o_da_progress_o_da_doen_a_de_parkinson_amp_parkinson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse projeto tem como objetivo usar dados disponibilizados no Kaggle para teste de de ferramentas de análise de dados, a fim de avaliar as medições de dados de proteínas e peptídeos de pacientes com doença de Parkinson para prever a progressão da doença. 

In [ ]:
#!pip install kaggle; #caso seja necessário atualizar a biblioteca no google colab

from google.colab import files
files.upload() #enviar o arquivo kaggle.json

#antes de importar o dataset nós iremos provisionar o local de armazenamento
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

#Aceitar a competição e copiar o endereço da API para o download do dataset
!kaggle competitions download -c amp-parkinsons-disease-progression-prediction
#Descompactar o arquivo baixado
!unzip \*.zip  && rm *.zip

Saving kaggle.json to kaggle.json
 68% 11.0M/16.1M [00:00<00:00, 98.7MB/s]
100% 16.1M/16.1M [00:00<00:00, 116MB/s] 
Archive:  amp-parkinsons-disease-progression-prediction.zip
  inflating: amp_pd_peptide/__init__.py  
  inflating: amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: example_test_files/sample_submission.csv  
  inflating: example_test_files/test.csv  
  inflating: example_test_files/test_peptides.csv  
  inflating: example_test_files/test_proteins.csv  
  inflating: public_timeseries_testing_util.py  
  inflating: supplemental_clinical_data.csv  
  inflating: train_clinical_data.csv  
  inflating: train_peptides.csv      
  inflating: train_proteins.csv      


In [ ]:
# importar os pacotes necessários
import pandas as pd

# importar o arquivo
df = pd.read_csv("train_clinical_data.csv") #dados clínicos

# ver 5 as primeiras entrada
df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


A pontuação do paciente para a parte N da Escala Unificada de Avaliação da Doença de Parkinson. Números mais altos indicam sintomas mais graves. Cada subseção abrange uma categoria distinta de sintomas, como humor e comportamento para a Parte 1 e funções motoras para a Parte 3.

In [ ]:
df.shape

(2615, 8)

In [ ]:
df['upd23b_clinical_state_on_medication']

0       NaN
1       NaN
2       NaN
3        On
4        On
       ... 
2610    Off
2611    Off
2612    Off
2613    Off
2614    Off
Name: upd23b_clinical_state_on_medication, Length: 2615, dtype: object

In [ ]:
df['upd23b_clinical_state_on_medication'].unique()

array([nan, 'On', 'Off'], dtype=object)

In [ ]:
df['visit_id']

0           55_0
1           55_3
2           55_6
3           55_9
4          55_12
          ...   
2610    65043_48
2611    65043_54
2612    65043_60
2613    65043_72
2614    65043_84
Name: visit_id, Length: 2615, dtype: object

Para retornar dados categóricos irei utilizar 'unique'

In [ ]:
df['visit_id'].unique()

array(['55_0', '55_3', '55_6', ..., '65043_60', '65043_72', '65043_84'],
      dtype=object)

In [ ]:
df['visit_month'].unique()

array([  0,   3,   6,   9,  12,  18,  24,  30,  36,  42,  48,  54,  60,
        72,  84,  96, 108])

In [ ]:
df['updrs_1'].unique()

array([10.,  8.,  7., 16., 14., 17., 12., 23.,  3.,  5.,  6.,  2.,  4.,
       11., 20., 19., 15.,  1.,  0.,  9., 13., 25., 21., 18., 26., 22.,
       27., 28., 31., 24., 29., nan, 33.])

In [ ]:
df['updrs_2'].unique()

array([ 6.,  7., 10.,  9., 13., 18., 20., 16., 21.,  2.,  5.,  3.,  4.,
        8., 11., 14., 15., 17.,  0.,  1., 12., 22., 19., 23., 26., 25.,
       24., 28., 27., 30., 31., 35., 40., 29., 39., nan, 36.])

In [ ]:
df['updrs_3'].unique()

array([15., 25., 34., 30., 41., 38., 49., 51., 52., 56., 20., 17., 21.,
       18., 23., 19., 32., 35., 26., 31., 33., 39., 28., 37., 40.,  0.,
        1.,  3.,  2.,  9., 14., 12., 11.,  7.,  6.,  8., 10., 36., 29.,
       22.,  5., 13.,  4., 45., 63., 50., 43., 46., 54., 24., 16., 27.,
       53., 47., 55., 68., nan, 59., 48., 44., 60., 64., 61., 42., 58.,
       66., 86., 85., 67., 57., 78., 62., 65.])

In [ ]:
df['updrs_4'].unique()

array([nan,  0.,  3.,  5.,  7., 10.,  8.,  4.,  6.,  2.,  9.,  1., 13.,
       11., 14., 12., 20., 15., 16., 17.])

In [ ]:
# importar os pacotes necessários
import pandas as pd

# importar o arquivo
df = pd.read_csv("train_peptides.csv") # nome do arquivo .csv que desejo importar (Dados de espectrometria de massa ao nível peptídico. Os peptídeos são as subunidades componentes das proteínas.)

# ver 5 as primeiras entrada
df.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


Para melhorar a visualização usei o "head" que mostra apenas os 5 primeiros resultados 


In [ ]:
df.shape

(981834, 6)

O dataframe contém um imenso número de linhas e 6 colunas

In [ ]:
df['visit_month']

0           0
1           0
2           0
3           0
4           0
         ... 
981829    108
981830    108
981831    108
981832    108
981833    108
Name: visit_month, Length: 981834, dtype: int64

In [ ]:
df['visit_month'].unique()

array([  0,   3,   6,  12,  18,  24,  30,  36,  48,  54,  60,  72,  84,
        96, 108])